In [2]:
import scipy.io as io
import torch 
import numpy as np

In [3]:
dset = io.loadmat('matlab_dataset/insect_dataset.mat')

In [4]:
all_dnas = dset['all_dnas']
all_labels = dset['all_labels']-1
all_dnas_torch = torch.load('tensor_dataset/all_dnas.pt')
all_labels_torch = torch.load('tensor_dataset/all_labels.pt')

train_loc = dset['train_loc']-1
val_seen_loc = dset['val_seen_loc']-1
val_seen_loc_torch = torch.load('tensor_dataset/val_seen_loc.pt').numpy()

val_unseen_loc = dset['val_unseen_loc']-1


In [5]:
for i,dna in enumerate(all_dnas):
    assert (dna == all_dnas_torch[i].numpy()).all()
    assert all_labels[0][i]== all_labels_torch[i],f"error with index {i}" 
    
    

In [6]:
for locmat,loctorch in zip(val_seen_loc[0],val_seen_loc_torch):
    assert(locmat==loctorch)

In [7]:
all_dnas = np.reshape(all_dnas,(32424,658*5))
all_dnas.shape
train_dnas = all_dnas[train_loc].squeeze()
train_labels = all_labels[0][train_loc].squeeze()

valseen_dnas = all_dnas[val_seen_loc].squeeze()
valseen_labels = all_labels[0][val_seen_loc].squeeze()

valunseen_dnas = all_dnas[val_unseen_loc].squeeze()
valunseen_labels = all_labels[0][val_unseen_loc].squeeze()

In [8]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(train_dnas, train_labels)

In [9]:
train_predicted_labels = clf.predict(train_dnas)
print(f"Training species accuracy:{np.count_nonzero(train_predicted_labels==train_labels)/len(train_labels)}")


Training species accuracy:0.9990796840248485


In [10]:
valseen_predicted_labels = clf.predict(valseen_dnas)
print(f"Validation seen accuracy:{np.count_nonzero(valseen_predicted_labels==valseen_labels)/len(valseen_labels)}")

Validation seen accuracy:0.9823747680890538


In [11]:
valunseen_predicted_labels = clf.predict(valunseen_dnas)
print(f"Validation unseen accuracy:{np.count_nonzero(valunseen_predicted_labels==valunseen_labels)/len(valunseen_labels)}")

Validation unseen accuracy:0.0


# prova features resnet101

In [12]:
resfeatures = io.loadmat('resnet_features.mat')

In [41]:
all_features = resfeatures['all_features']
train_features = all_features[train_loc[0,:],:]
val_seen_features = all_features[val_seen_loc[0,:],:]
val_unseen_features = all_features[val_unseen_loc[0,:],:]

In [38]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(min_samples_leaf=2,n_jobs=-1)
clf = clf.fit(train_features, train_labels)

In [39]:
train_predicted_labels = clf.predict(train_features)
print(f"Training species accuracy:{np.count_nonzero(train_predicted_labels==train_labels)/len(train_labels)}")


Training species accuracy:0.9996932280082829


In [45]:
valseen_predicted_labels = clf.predict(val_seen_features)
print(f"Validation seen accuracy:{np.count_nonzero(valseen_predicted_labels==valseen_labels)/len(valseen_labels)}")

Validation seen accuracy:0.3002473716759431


In [42]:
valunseen_predicted_labels = clf.predict(val_unseen_features)
print(f"Validation unseen accuracy:{np.count_nonzero(valunseen_predicted_labels==valunseen_labels)/len(valunseen_labels)}")

Validation unseen accuracy:0.0


In [47]:
valseen_predicted_labels = clf.predict(np.concatenate((val_seen_features,val_unseen_features)))
print(f"Validation seen+unseen accuracy:{np.count_nonzero(
    valseen_predicted_labels==np.concatenate((valseen_labels,valunseen_labels)))/len(np.concatenate((valseen_labels,valunseen_labels)))}")


Validation seen+unseen accuracy:0.1396117900790798
